<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/Drug_Data_Mapping_and_Write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Read into one dataset and get its indexes

In [1]:
import chardet
with open('CHLDiab2.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'ISO-8859-1', 'language': ''}

In [2]:
import pandas as pd
DataSet = pd.read_csv('CHLDiab2.csv',encoding='ISO-8859-1')
DataSet.head(1)

,Unnamed: 0,Codigo,Tipo de Adquisicion,Estado,Nombre Organismo,Region Unidad,Codigo Moneda,Fecha Creacion,Fecha Publicacion,Fecha Adjudicacion,...,Valor Total Ofertado,Cantidad Adjudicada,Monto Linea Adjudica,Fecha Envio Oferta,Oferta seleccionada,CHL-USD,Monto Unitario Oferta USD,Monto Linea Adjudica USD,Valor Total Ofertado USD,Tipo de Institucion
0,1,8636737,Licitación Pública Entre 100 y 1000 UTM (LE),8,I MUNICIPALIDAD DE CHIGUAYANTE,Región del Biobío,CLP,2020-01-07,2020-01-10,2020-01-24,...,516000.0,1200.0,516000.0,2020-01-15,Seleccionada,773.730435,0.555749,666.898931,666.898931,Municipalidad


In [3]:
DrugNames = DataSet['Descripcion linea Adquisicion']

In [4]:
len(DrugNames) # length of drug names

5331

In [5]:
DrugNames = list(set(DrugNames))
len(DrugNames) # length of drug names after remove duplicates

952

In [6]:
# uppercase everything
DrugNamesU = []
for i in DrugNames:
  DrugNamesU.append(i.upper())

In [7]:
DrugNamesU[:5]

['GLIBENCLAMIDA 5 MG.',
 'GLIBENCLAMIDA 5MG. CON FECHA VENCIMIENTO ENERO 2023.',
 'GALVUS 50 MG, CAJA DE 56 COMPRIMIDOS',
 'GLIBENCLAMIDA 5 MG. COMPRIMIDO.  INDICAR FECHA DE VENCIMIENTO EN LA PRESENTACIÓN DE LOS PRODUCTOS.',
 'GLIBENCLAMIDA 5 MG EN COMPRIMIDOS']

In [8]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = DrugNamesU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.4 MB 5.4 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=af9be5c4c3d567d07e5fe49d04283c43d995114875fb6f0ffa592af908d7c676
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
GLIBENCLAMIDA 5 MG.


In [9]:
DrugNamesU[0]

'GLIBENCLAMIDA 5 MG.'

In [10]:
DrugNamesUEng = []
for item in DrugNamesU:
  translation = translator.translate(item, dest='en')
  DrugNamesUEng.append(translation.text)
# runs 1 mins

In [11]:
DrugNamesUEng[:5]

['GLIBENCLAMIDA 5 MG.',
 'GLYBENCLAMIDE 5MG. WITH DUE DATE JANUARY 2023.',
 'GALVUS 50 MG, BOX OF 56 TABLETS',
 'GLYBENCLAMIDE 5 MG. COMPRESSED. INDICATE EXPIRATION DATE IN THE PRESENTATION OF THE PRODUCTS.',
 'GLIBENCLAMIDE 5 MG TABLETS']

In [12]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

DrugNamesUEngSpl = []
for item in DrugNamesUEng:
  tokenlist = tokenizer.tokenize(item)
  DrugNamesUEngSpl.append(tokenlist)

In [13]:
DrugNamesUEngSpl[:1]

[['GLIBENCLAMIDA', '5', 'MG', '.']]

In [14]:
DrugNamesIndex  = []
for item in DrugNamesUEngSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  DrugNamesIndex.append(templist)

In [15]:
len(DrugNamesIndex)

952

In [16]:
DrugNamesIndex[:5]

[['GLIBENCLAMIDA'],
 ['GLYBENCLAMIDE', 'JANUARY'],
 ['GALVUS', 'TABLETS'],
 ['GLYBENCLAMIDE',
  'COMPRESSED',
  'INDICATE',
  'EXPIRATION',
  'PRESENTATION',
  'PRODUCTS'],
 ['GLIBENCLAMIDE', 'TABLETS']]

In [17]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [18]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [19]:
DrugNamesIndexDelSIMILAR = []
for item in DrugNamesIndex:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  DrugNamesIndexDelSIMILAR.append(add)

In [20]:
DrugNamesIndexDelSIMILAR[:5]

[['GLIBENCLAMIDA'],
 ['GLYBENCLAMIDE', 'JANUARY'],
 ['GALVUS', 'TABLETS'],
 ['GLYBENCLAMIDE',
  'COMPRESSED',
  'INDICATE',
  'EXPIRATION',
  'PRESENTATION',
  'PRODUCTS'],
 ['GLIBENCLAMIDE', 'TABLETS']]

## Part 2: Set up indexes for clean catagol drug names

In [21]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [22]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [23]:
CatalogNameUS[:5] # after split by ","

[['METFORMIN 1000 MG + GLIMEPIRIDE 4 MG', ' TABLET', ' TBD'],
 ['CYCLOSPORINE 10 MG', ' TBD', ' TBD'],
 ['ETOPOSIDE 20 MG/ML', ' INJECTION', ' 5 ML AMPOULE', ' 1X1'],
 ['EFAVIRENZ 600 MG + EMTRICITABINE 200 MG + TENOFOVIR 300 MG',
  ' TABLET',
  ' 30 TABLETS BOTTLE'],
 ['SOFOSBUVIR 400 MG', ' TABLET', ' 28 TABLETS BOTTLE']]

In [24]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [25]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [26]:
PresentationForm[:5]# the presentation form

[' CAPSULE (PRONLONGED RELEASE)',
 ' ORAL SUSPENSION',
 ' TBD',
 ' 4MG TABLET',
 ' INJECTION. 500MG/20ML ROAD BOTTLE']

In [27]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [28]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [29]:
PresentationDiction2 = list(set(PresentationDiction2))

In [30]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [31]:
print(len(MainMedicine), MainMedicine[:5])

462 ['METFORMIN 1000 MG + GLIMEPIRIDE 4 MG', 'CYCLOSPORINE 10 MG', 'ETOPOSIDE 20 MG/ML', 'EFAVIRENZ 600 MG + EMTRICITABINE 200 MG + TENOFOVIR 300 MG', 'SOFOSBUVIR 400 MG']


In [32]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [33]:
MainMedicineSpl[:5]

[['METFORMIN', '1000', 'MG', '+', 'GLIMEPIRIDE', '4', 'MG'],
 ['CYCLOSPORINE', '10', 'MG'],
 ['ETOPOSIDE', '20', 'MG', '/', 'ML'],
 ['EFAVIRENZ',
  '600',
  'MG',
  '+',
  'EMTRICITABINE',
  '200',
  'MG',
  '+',
  'TENOFOVIR',
  '300',
  'MG'],
 ['SOFOSBUVIR', '400', 'MG']]

In [34]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)

In [35]:
MainMedicineIndex[:5]

[['METFORMIN', 'GLIMEPIRIDE'],
 ['CYCLOSPORINE'],
 ['ETOPOSIDE'],
 ['EFAVIRENZ', 'EMTRICITABINE', 'TENOFOVIR'],
 ['SOFOSBUVIR']]

In [36]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [37]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [38]:
len(CatalogIndexData_df)

462

In [39]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"METFORMIN 1000 MG + GLIMEPIRIDE 4 MG, TABLET, TBD","[METFORMIN, GLIMEPIRIDE]"
1,"CYCLOSPORINE 10 MG, TBD, TBD",[CYCLOSPORINE]
2,"ETOPOSIDE 20 MG/ML, INJECTION, 5 ML AMPOULE, 1X1",[ETOPOSIDE]
3,EFAVIRENZ 600 MG + EMTRICITABINE 200 MG + TENO...,"[EFAVIRENZ, EMTRICITABINE, TENOFOVIR]"
4,"SOFOSBUVIR 400 MG, TABLET, 28 TABLETS BOTTLE",[SOFOSBUVIR]


In [40]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [41]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [42]:
CatalogMedName[:3]

['METHYLDOPA', 'DACARBAZINE', 'VORICONAZOLE']

## Part 3: Write the Chile 2 Index Data Set According to the Catalog Name Index

In [43]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 2 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [44]:
finIndex = []
for sublist in DrugNamesIndexDelSIMILAR:
  add = KeepMetAnd(sublist)
  finIndex.append(add)

In [45]:
# count the number of Chile index sub list which is null []
count = 0
for i in finIndex:
  if not i:
    count += 1

In [46]:
print(len(finIndex), count) # the total number of drug names and the names with null index

952 427


In [47]:
nullindexlist = []
for i in range(0, len(finIndex)):
  if not finIndex[i]:
    nullindexlist.append(i)

In [48]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 13.5 MB 6.6 MB/s 
     |████████████████████████████████| 71 kB 10.1 MB/s 
     |████████████████████████████████| 6.0 MB 44.6 MB/s 
     |████████████████████████████████| 188 kB 53.4 MB/s 
     |████████████████████████████████| 10.1 MB 52.6 MB/s 
     |████████████████████████████████| 660 kB 52.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found ex

In [49]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.7 MB/s 
     |████████████████████████████████| 5.8 MB 2.5 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=0a7e3731b7eccee821a4d3c4924506502740588d50ec33a3b4cdc41ef48bf254
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scisp

In [50]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [51]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

In [52]:
drugOutDataName = []
for nullindex in nullindexlist:
  tempChiledrugname = DrugNamesIndexDelSIMILAR[nullindex]
  tempChiledrugname_str = str(' '.join(tempChiledrugname))
  doc = nlp(tempChiledrugname_str)
  temptext = doc.text
  drugOutDataName.append(temptext)

In [53]:
drugOutDataNameNullNumber = 0

for i in drugOutDataName:
  if not i:
    drugOutDataNameNullNumber += 1

In [54]:
drugOutDataNameNullNumber

2

In [55]:
# for these null element in CHIfnIndex list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  finIndex[i] = drugOutDataName[j]
  j += 1

## Part 4: Use SimHash select cloest index for drug names

In [56]:
!pip install simhash

from simhash import Simhash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
pairIndexlist = []
for dirtyname in finIndex:
  distance_list = []
  for index in MainMedicineIndex:
    distance = Simhash(index).distance(Simhash(dirtyname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__) # get the index which correspond to the minimum distance
  add_index = MainMedicineIndex[index_min]
  pairIndexlist.append(add_index)

In [58]:
PairIndexData = {"drug names": finIndex, "Catalog Index": pairIndexlist}

In [59]:
print(len(finIndex), len(pairIndexlist))

952 952


In [60]:
PairIndexData_df = pd.DataFrame(PairIndexData)

In [61]:
PairIndexData_df

,drug names,Catalog Index
0,GLIBENCLAMIDA,"[IRBESARTAN, HYDROCHLOROTHIAZIDE]"
1,GLYBENCLAMIDE JANUARY,[ITRACONAZOLE]
2,GALVUS TABLETS,"[MEGLUMINE, ANTIMONIATE]"
3,GLYBENCLAMIDE COMPRESSED INDICATE EXPIRATION P...,"[VILDAGLIPTIN, METFORMIN]"
4,[GLIBENCLAMIDE],[GLIBENCLAMIDE]
...,...,...
947,METFORMINA 500MG GLIBENCLAMIDA,[PROPOFOL]
948,"[GLIBENCLAMIDE, METFORMIN]","[GLIBENCLAMIDE, METFORMIN]"
949,METFORMINA GLIBENCLAMIDA CARDIOVASCULAR,"[MEGLUMINE, ANTIMONIATE]"
950,GLIBENCLAMIDA,"[IRBESARTAN, HYDROCHLOROTHIAZIDE]"


## Part 5: combine the Chile drug name index pair data frame and full catagol name index data frame to get the result

In [62]:
# for Chile pair index dataframe, transform list to string
str_pairIndexlist = []
for item in pairIndexlist:
  newitem = ' '.join(item)
  str_pairIndexlist.append(newitem)

In [63]:
str_finIndex = []
for item in finIndex:
  newitem = ''.join(item)
  str_finIndex.append(newitem)

In [64]:
newPairIndexData = {"drug names": str_finIndex, "Catalog Index": str_pairIndexlist}

In [65]:
newPairIndexData_df = pd.DataFrame(newPairIndexData)

In [66]:
newPairIndexData_df[:2]

,drug names,Catalog Index
0,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE
1,GLYBENCLAMIDE JANUARY,ITRACONAZOLE


In [67]:
# for catalog name index dataframe, transform list to string
str_MainMedicineIndex = []
for item in MainMedicineIndex:
  newitem = ' '.join(item)
  str_MainMedicineIndex.append(newitem)

In [68]:
newCatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": str_MainMedicineIndex}

In [69]:
newCatalogIndexData_df = pd.DataFrame(newCatalogIndexData)

In [70]:
newCatalogIndexData_df[:2]

,Catalog Name,Catalog Index
0,"METFORMIN 1000 MG + GLIMEPIRIDE 4 MG, TABLET, TBD",METFORMIN GLIMEPIRIDE
1,"CYCLOSPORINE 10 MG, TBD, TBD",CYCLOSPORINE


In [71]:
combine_pair_df = pd.merge(newPairIndexData_df, newCatalogIndexData_df, on='Catalog Index', how='left')

## Part 6: Combine the original drug name in Chile 1 with the pairing data frame we have got, then select for the catalog name second time (compute the distance between the original drug name and the sub set of catalog under each name in our result)

In [72]:
print(len(DrugNames), len(str_finIndex))

952 952


In [73]:
# combine uppercase english chile 1 drug name with its indexes
drugfn_index = {"drugs english uppercase names": DrugNamesUEng, "drug names": str_finIndex}

In [74]:
hile1drugfn_index_df = pd.DataFrame(drugfn_index)

In [75]:
hile1drugfn_index_df[:5]

,drugs english uppercase names,drug names
0,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA
1,GLYBENCLAMIDE 5MG. WITH DUE DATE JANUARY 2023.,GLYBENCLAMIDE JANUARY
2,"GALVUS 50 MG, BOX OF 56 TABLETS",GALVUS TABLETS
3,GLYBENCLAMIDE 5 MG. COMPRESSED. INDICATE EXPIR...,GLYBENCLAMIDE COMPRESSED INDICATE EXPIRATION P...
4,GLIBENCLAMIDE 5 MG TABLETS,GLIBENCLAMIDE


In [76]:
# combine Chile drug original full name, its index with the result data frame (pair with catalog name)
drugs = pd.merge(hile1drugfn_index_df, combine_pair_df, on='drug names', how='left')

In [77]:
drugs[:5]

,drugs english uppercase names,drug names,Catalog Index,Catalog Name
0,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE,"IRBESARTAN 150 MG + HYDROCHLOROTHIAZIDE 25 MG,..."
1,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,..."
2,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,..."
3,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE,"IRBESARTAN 150 MG + HYDROCHLOROTHIAZIDE 25 MG,..."
4,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,IRBESARTAN HYDROCHLOROTHIAZIDE,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,..."


In [78]:
print(len(drugs), len(set(drugs['drugs english uppercase names'])))

91484 841


In [79]:
# get the list that contains first pairing names list for each original drug full name
fistpairinglist = []
i = 0
for name in DrugNamesUEng:
  subpairlist = []
  while i < len(drugs):
    if drugs['drugs english uppercase names'][i] == name:
      subpairlist.append(drugs['Catalog Name'][i])
      i += 1
    else:
      break
  fistpairinglist.append(subpairlist)
# runs 4 mins

In [80]:
print(len(fistpairinglist), len(DrugNamesUEng))

952 952


In [81]:
for i in range(0, len(fistpairinglist)):
  if not fistpairinglist[i]:
    fistpairinglist[i] = DrugNamesUEng[i]

In [82]:
finalpairname = []
for item in range(len(DrugNamesUEng)):
  distance_list = []
  for pairname in fistpairinglist[item]:
    distance = Simhash(DrugNamesUEng[item]).distance(Simhash(pairname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__)
  add_index = fistpairinglist[item][index_min]
  finalpairname.append(add_index)

In [83]:
# combine the final paired name with the Chile 1 drug english name
drugfn_index = {"drugs english uppercase names": DrugNamesUEng, "Final paired catalog name": finalpairname}

In [84]:
len(finalpairname)

952

In [85]:
len(DrugNamesUEng)

952

In [86]:
drugfn_index_df = pd.DataFrame(drugfn_index)


In [113]:
drugfn_index_df

,drugs english uppercase names,Final paired catalog name
0,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,..."
1,GLYBENCLAMIDE 5MG. WITH DUE DATE JANUARY 2023.,"ITRACONAZOLE 100 MG, CAPSULE, BLISTER, 5X3"
2,"GALVUS 50 MG, BOX OF 56 TABLETS","MEGLUMINE ANTIMONIATE 300 MG/ML, INJECTION, 5 ..."
3,GLYBENCLAMIDE 5 MG. COMPRESSED. INDICATE EXPIR...,"VILDAGLIPTIN 50 MG + METFORMIN 500 MG, TABLET,..."
4,GLIBENCLAMIDE 5 MG TABLETS,"GLIBENCLAMIDE 5 MG, TABLET, TBD"
...,...,...
947,METFORMINA 500MG/ GLIBENCLAMIDA 5MG X 60 COMP,"PROPOFOL 20 MG/ML, TBD, TBD"
948,GLIBENCLAMIDE/METFORMIN 5MG./500MG CM,"GLIBENCLAMIDE 5 MG + METFORMIN 1000 MG, TABLET..."
949,METFORMINA + GLIBENCLAMIDA 500/2.5MG **CARDIOV...,"MEGLUMINE ANTIMONIATE 300 MG/ML, INJECTION, 5 ..."
950,50-214-700-004-00\tGLIBENCLAMIDA 5MG,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,..."


In [88]:
#drugfn_index_df.to_excel("Chile2drugfn_index.xlsx", sheet_name = 'Chile2drugfn_index chile 1')  

## Part 7: Combine (Add) the write result 

In [ ]:
#finIndex

In [89]:
PreList = []

for item in DrugNamesUEngSpl:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  PreList.append(list(set(addlist)))

In [90]:
WeightList = []

for item in DrugNamesUEngSpl:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [91]:
print(len(finIndex), len(PreList), len(WeightList))

952 952 952


In [93]:
# for catalog name index dataframe, transform list to string
str_finIndex = []
for item in finIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_finIndex.append(newitem)

In [94]:
str_finIndex[:4]

['GLIBENCLAMIDA',
 'GLYBENCLAMIDE JANUARY',
 'GALVUS TABLETS',
 'GLYBENCLAMIDE COMPRESSED INDICATE EXPIRATION PRESENTATION PRODUCTS']

In [95]:
str_PreList = []

for item in PreList:
  newitem = ' '.join(item)
  str_PreList.append(newitem)

In [96]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [104]:
DrugDataWrite = {"drugs english uppercase names": DrugNamesUEng, "Final Drug Index": str_finIndex, "Presentation Form": str_PreList, "Weight": str_WeightList}

In [105]:
DrugDataWrite_df = pd.DataFrame(DrugDataWrite)

In [106]:
DrugDataWrite_df

,drugs english uppercase names,Final Drug Index,Presentation Form,Weight
0,GLIBENCLAMIDA 5 MG.,GLIBENCLAMIDA,,MG 5
1,GLYBENCLAMIDE 5MG. WITH DUE DATE JANUARY 2023.,GLYBENCLAMIDE JANUARY,,
2,"GALVUS 50 MG, BOX OF 56 TABLETS",GALVUS TABLETS,,MG 50
3,GLYBENCLAMIDE 5 MG. COMPRESSED. INDICATE EXPIR...,GLYBENCLAMIDE COMPRESSED INDICATE EXPIRATION P...,,MG 5
4,GLIBENCLAMIDE 5 MG TABLETS,GLIBENCLAMIDE,,MG 5
...,...,...,...,...
947,METFORMINA 500MG/ GLIBENCLAMIDA 5MG X 60 COMP,METFORMINA 500MG GLIBENCLAMIDA,500MG,
948,GLIBENCLAMIDE/METFORMIN 5MG./500MG CM,GLIBENCLAMIDE METFORMIN,500MG,
949,METFORMINA + GLIBENCLAMIDA 500/2.5MG **CARDIOV...,METFORMINA GLIBENCLAMIDA CARDIOVASCULAR,,
950,50-214-700-004-00\tGLIBENCLAMIDA 5MG,GLIBENCLAMIDA,,


In [114]:
drugs_write_map = pd.merge(drugfn_index_df, DrugDataWrite_df, on='drugs english uppercase names')

In [115]:
drugs_write_map

,drugs english uppercase names,Final paired catalog name,Final Drug Index,Presentation Form,Weight
0,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,MG 5
1,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,MG 5
2,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,MG 5
3,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,MG 5
4,GLIBENCLAMIDA 5 MG.,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,MG 5
...,...,...,...,...,...
1415,METFORMINA 500MG/ GLIBENCLAMIDA 5MG X 60 COMP,"PROPOFOL 20 MG/ML, TBD, TBD",METFORMINA 500MG GLIBENCLAMIDA,500MG,
1416,GLIBENCLAMIDE/METFORMIN 5MG./500MG CM,"GLIBENCLAMIDE 5 MG + METFORMIN 1000 MG, TABLET...",GLIBENCLAMIDE METFORMIN,500MG,
1417,METFORMINA + GLIBENCLAMIDA 500/2.5MG **CARDIOV...,"MEGLUMINE ANTIMONIATE 300 MG/ML, INJECTION, 5 ...",METFORMINA GLIBENCLAMIDA CARDIOVASCULAR,,
1418,50-214-700-004-00\tGLIBENCLAMIDA 5MG,"IRBESARTAN 300 MG + HYDROCHLOROTHIAZIDE 25 MG,...",GLIBENCLAMIDA,,


In [124]:
drugs_write_map = drugs_write_map.drop_duplicates(subset=['drugs english uppercase names'], keep='first')

In [117]:
len(DrugNamesUEng)

952

In [118]:
len(list(DrugNamesUEng))

952

In [125]:
drugs_write_map.to_excel("CHLDiab2drugs_write_map.xlsx", sheet_name = 'CHLDiab2drugs_write_map')  